In [1]:
import numpy as np
import os
import glob
import topofisher
from topofisher.filtrations.numpy.alphaDTML import AlphaDTMLayer
from topofisher.filtrations.numpy.default_filtrations import AlphaFiltration
from topofisher.vectorizations.numpy.custom_vectorizations import TOPK
from topofisher.vectorizations.numpy.vectorization_layer import VectorizationLayers
import matplotlib.pyplot as plt
from topofisher.fisher.Fisher import show_fm_and_bias, fisherMOPED, fisherFromVecs
import tensorflow as tf
import importlib
importlib.reload(topofisher.fisher.imnn)
from topofisher.fisher.imnn import IMNNLayer, MopedLayer, ExtraDimLayer
import pickle, glob
import numpy as np 
import CompressedFisher

2023-10-30 21:37:56.765645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 21:38:03.882733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## For power spectrum 

In [10]:
fid_arr = np.load('/Users/karthikviswanathan/Downloads/galaxies_Pk_combined_P0kP2kB0k/fiducial_P0kP2kB0k.npy')
parameter_names = ['alpha_m', 'sigma_logM_m', 'logM0_m', 'logM1_m', 'logMmin_m', 'ns_m', \
                      'Ob2_m', 'Om_m', 's8_m', 'h_m', 'OR_LSS_m', 'LC_m', 'EQ_m']
params_fid =  [1.1, 0.2, 14, 14, 13.65, 0.9624, 0.049, 0.3175, 0.834, 0.6711, 0., 0., 0.]
der_arr = []
for item in parameter_names:
    filename = '/Users/karthikviswanathan/Downloads/galaxies_Pk_deriv_P0kP2kB0k/' + item + '_P0kP2kB0k_deriv.npy'
    arr = np.load(filename)
    # print(arr.shape)
    der_arr.append(arr)


In [3]:
compress_frac_split = 0.5
# dict_param_steps = {parameter_names[i]:delta_params[i] for i in range(len(params_fid))}
dict_deriv_sims = {parameter_names[i]:der_arr[i] for i in range(len(params_fid))}
nSimsCovMat = 10000
nSims_deriv = 7500
cFisher = CompressedFisher.gaussianFisher(parameter_names, nSims_deriv, \
                                          include_covmat_param_depedence=False)
covmat_sims = fid_arr
cFisher.initailize_covmat(covmat_sims)
cFisher.initailize_mean(covmat_sims)
cFisher.initailize_deriv_sims(dic_deriv_sims=dict_deriv_sims)
cFisher.generate_deriv_sim_splits(compress_frac_split)

stnd_constraint    = cFisher.compute_fisher_forecast(parameter_names)
stnd_constraint_bias = cFisher.est_fisher_forecast_bias(parameter_names)

print(f'Parameter  \t standard Fisher \t Est. Fractional bias ')
for i,name in enumerate(parameter_names):
    print(f'{name}  \t\t  {stnd_constraint[i,i]**.5:.3f} \t\t     {(stnd_constraint_bias/stnd_constraint)[i,i]:.3f} ')



Parameter  	 standard Fisher 	 Est. Fractional bias 
alpha_m  		  0.373 		     0.329 
sigma_logM_m  		  0.146 		     0.731 
logM0_m  		  0.600 		     0.358 
logM1_m  		  0.171 		     0.211 
logMmin_m  		  0.231 		     0.369 
ns_m  		  0.043 		     0.306 
Ob2_m  		  0.004 		     0.275 
Om_m  		  0.012 		     0.163 
s8_m  		  0.015 		     0.194 
h_m  		  0.044 		     0.352 
OR_LSS_m  		  91.283 		     0.119 
LC_m  		  59.364 		     0.041 
EQ_m  		  229.629 		     0.243 


In [4]:
compressed_constraint = cFisher.compute_compressed_fisher_forecast(parameter_names)
compressed_constraint_bias = cFisher.est_compressed_fisher_forecast_bias(parameter_names)

print(f'Parameter \t Compressed Fisher \t Est. Fractional bias ')
for i,name in enumerate(parameter_names):
    print(f'{name}   \t\t  {compressed_constraint[i,i]**.5:.3f} \t\t   \
    {(compressed_constraint_bias/compressed_constraint)[i,i]:.3f} ')


combined_constraint = cFisher.compute_combined_fisher_forecast(parameter_names)
# print(f'Parameter  standard Fisher  Bias compressed comp Bias combined ')
print(f'Parameter \t Combined Fisher  ')
for i,name in enumerate(parameter_names):
    print(f'{name}   \t\t  {combined_constraint[i,i]**.5:.3f} ')

Parameter 	 Compressed Fisher 	 Est. Fractional bias 
alpha_m   		  0.555 		       0.010 
sigma_logM_m   		  0.468 		       0.086 
logM0_m   		  0.915 		       0.010 
logM1_m   		  0.225 		       0.007 
logMmin_m   		  0.464 		       0.060 
ns_m   		  0.064 		       0.009 
Ob2_m   		  0.006 		       0.009 
Om_m   		  0.015 		       0.006 
s8_m   		  0.019 		       0.007 
h_m   		  0.068 		       0.012 
OR_LSS_m   		  107.073 		       0.005 
LC_m   		  62.470 		       0.002 
EQ_m   		  305.421 		       0.006 
Parameter 	 Combined Fisher  
alpha_m   		  0.452 
sigma_logM_m   		  0.261 
logM0_m   		  0.739 
logM1_m   		  0.195 
logMmin_m   		  0.312 
ns_m   		  0.053 
Ob2_m   		  0.005 
Om_m   		  0.013 
s8_m   		  0.016 
h_m   		  0.054 
OR_LSS_m   		  98.032 
LC_m   		  60.752 
EQ_m   		  263.696 


In [8]:
for i,name in enumerate(parameter_names):
    print(f'{name}   \t\t  {compressed_constraint[i,i]**.5:.4f} \t\t   \
    {(compressed_constraint_bias/compressed_constraint)[i,i]:.4f} ')

 

alpha_m   		  0.5546 		       0.0098 
sigma_logM_m   		  0.4684 		       0.0861 
logM0_m   		  0.9147 		       0.0102 
logM1_m   		  0.2253 		       0.0069 
logMmin_m   		  0.4642 		       0.0605 
ns_m   		  0.0642 		       0.0091 
Ob2_m   		  0.0060 		       0.0093 
Om_m   		  0.0149 		       0.0060 
s8_m   		  0.0188 		       0.0071 
h_m   		  0.0675 		       0.0115 
OR_LSS_m   		  107.0725 		       0.0047 
LC_m   		  62.4705 		       0.0015 
EQ_m   		  305.4215 		       0.0062 


In [9]:
print(f'Parameter  \t standard Fisher \t Est. Fractional bias ')
for i,name in enumerate(parameter_names):
    print(f'{name}  \t\t  {stnd_constraint[i,i]**.5:.4f} \t\t     {(stnd_constraint_bias/stnd_constraint)[i,i]:.3f} ')


Parameter  	 standard Fisher 	 Est. Fractional bias 
alpha_m  		  0.3728 		     0.329 
sigma_logM_m  		  0.1458 		     0.731 
logM0_m  		  0.6002 		     0.358 
logM1_m  		  0.1715 		     0.211 
logMmin_m  		  0.2312 		     0.369 
ns_m  		  0.0435 		     0.306 
Ob2_m  		  0.0043 		     0.275 
Om_m  		  0.0118 		     0.163 
s8_m  		  0.0146 		     0.194 
h_m  		  0.0440 		     0.352 
OR_LSS_m  		  91.2828 		     0.119 
LC_m  		  59.3640 		     0.041 
EQ_m  		  229.6291 		     0.243 


In [ ]:
Om    0.013
Ob2    0.005 
h      0.054
ns     0.053 
s8     0.016 